In [2]:
import tarfile
import pandas as pd
import numpy as np

In [24]:
import os, shutil

In [7]:
def handle_tarfile(filePath):
    if filePath.endswith('tar.gz'):
        tarContent = tarfile.open(filePath, "r:gz")
    else:
        tarContent = tarfile.open(filePath, "r:")
        
    tarContent.extractall()
    tarContent.close()

In [8]:
handle_tarfile('CrisisMMD_v1.0.tar.gz')

In this project we are interested in assessing the damage caused to properties during natural disasters. We are going to do image classification to solve the problem.

let us read one of the tsv files from the annotations folder in order to understand the contents of the dataset and how to go ahead with solving the problem

In [12]:
example_df = pd.read_csv('annotations/hurricane_harvey_final_data.tsv', sep='\t')

In [13]:
example_df.head(10)

,tweet_id,image_id,text_info,text_info_conf,image_info,image_info_conf,text_human,text_human_conf,image_human,image_human_conf,image_damage,image_damage_conf,tweet_text,image_url,image_path
0,905274232590004225,901671684478029825_0,not_informative,1.0000,informative,1.0000,not_relevant_or_cant_judge,1.0000,infrastructure_and_utility_damage,1.0000,severe_damage,1.0000,"CONGRATS ON HITTING YOIR GOAL GUYS, I'm sure t...",http://pbs.twimg.com/media/DINhGs8XcAE1H7q.jpg,data_image/hurricane_harvey/27_8_2017/90167168...
1,901646074527535105,901646074527535105_0,informative,0.6822,informative,1.0000,injured_or_dead_people,0.6822,infrastructure_and_utility_damage,1.0000,severe_damage,0.6728,RT @ajwamood: #ajwamood : Harvey the first maj...,http://pbs.twimg.com/media/DILxh_xWAAAfJDY.jpg,data_image/hurricane_harvey/27_8_2017/90164607...
2,901646074527535105,901646074527535105_1,informative,0.6822,informative,1.0000,injured_or_dead_people,0.6822,infrastructure_and_utility_damage,1.0000,severe_damage,1.0000,RT @ajwamood: #ajwamood : Harvey the first maj...,http://pbs.twimg.com/media/DILxiisXYAAokz_.jpg,data_image/hurricane_harvey/27_8_2017/90164607...
3,901646074527535105,901646074527535105_2,informative,0.6822,informative,1.0000,injured_or_dead_people,0.6822,infrastructure_and_utility_damage,1.0000,severe_damage,0.6528,RT @ajwamood: #ajwamood : Harvey the first maj...,http://pbs.twimg.com/media/DILxjSUWAAEIaWH.jpg,data_image/hurricane_harvey/27_8_2017/90164607...
4,901646074527535105,901646074527535105_3,informative,0.6822,informative,1.0000,injured_or_dead_people,0.6822,infrastructure_and_utility_damage,1.0000,severe_damage,0.6812,RT @ajwamood: #ajwamood : Harvey the first maj...,http://pbs.twimg.com/media/DILxkWJXgAANF-E.jpg,data_image/hurricane_harvey/27_8_2017/90164607...
5,901646123080830976,901646123080830976_0,not_informative,1.0000,not_informative,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,RT @yIIeza: When we get back to SCHS after Har...,http://pbs.twimg.com/media/DIMk2e0UQAAUk2B.jpg,data_image/hurricane_harvey/27_8_2017/90164612...
6,901646123080830976,901646123080830976_1,not_informative,1.0000,not_informative,0.3429,NaN,NaN,NaN,NaN,NaN,NaN,RT @yIIeza: When we get back to SCHS after Har...,http://pbs.twimg.com/media/DIMk2eyUQAAywFD.jpg,data_image/hurricane_harvey/27_8_2017/90164612...
7,901646123080830976,901646123080830976_2,not_informative,1.0000,not_informative,0.6871,NaN,NaN,NaN,NaN,NaN,NaN,RT @yIIeza: When we get back to SCHS after Har...,http://pbs.twimg.com/media/DIMk2exV4AEmQRm.jpg,data_image/hurricane_harvey/27_8_2017/90164612...
8,901646123080830976,901646123080830976_3,informative,0.6814,informative,1.0000,other_relevant_information,0.6814,infrastructure_and_utility_damage,1.0000,severe_damage,0.6866,RT @yIIeza: When we get back to SCHS after Har...,http://pbs.twimg.com/media/DIMk2exV4AArVwo.jpg,data_image/hurricane_harvey/27_8_2017/90164612...
9,901646127895863296,901646127895863296_0,informative,0.9167,informative,0.9257,other_relevant_information,0.9167,other_relevant_information,0.9257,NaN,NaN,Not always good when your city shows up on a s...,http://pbs.twimg.com/media/DINKP-iVwAIIRJi.jpg,data_image/hurricane_harvey/27_8_2017/90164612...


In [16]:
#checking the number of records
len(example_df)

4443

We are particularly interested in the "image_damage" and "image_path" columns for the current project

In [17]:
example_df['image_damage'].value_counts()

severe_damage              556
mild_damage                222
little_or_no_damage        116
dont_know_or_cant_judge     22
Name: image_damage, dtype: int64

for the current project, we will be interested in records which have a value for "image_damage" and filter out records which have an empty or NaN value for "image_damage"

Let us count the number of NaN values in "image_damage" column

In [18]:
example_df['image_damage'].isna().sum()

3527

We will be following the below approach:
1. Let us create a main folder (all_disaster_images) and inside that 3 folders (severe_damage, mild_damage, little_or_no_damage), which will contain the images pertaining to each severity of damage
2. We will go through all the tsv files pertaining to different natural disasters in the annotations folder
3. Records which have valid "image_damage" values, we will copy the image from the "image_path" and paste it in one of the 3 folders created above based on the "image_damage" type 

In [21]:
severity_levels = ["severe_damage", "mild_damage", "little_or_no_damage"]

In [22]:
base_dir = 'all_disaster_images'
os.mkdir(base_dir)

In [23]:
for cur_level in severity_levels:
    cur_dir = os.path.join(base_dir, cur_level)
    os.mkdir(cur_dir)

In [25]:
tsv_files = os.listdir('annotations/')

In [34]:
for cur_file in tsv_files:
    print('current tsv file:', cur_file)
    cur_df = pd.read_csv('annotations/' + cur_file, sep='\t')
    print('number of records', len(cur_df))
    print('counts\n', cur_df['image_damage'].value_counts())
    print('number of NaN records', cur_df['image_damage'].isna().sum())
    print("===========================================================")

current tsv file: california_wildfires_final_data.tsv
number of records 1589
counts
 severe_damage              465
mild_damage                 51
little_or_no_damage         15
dont_know_or_cant_judge     14
Name: image_damage, dtype: int64
number of NaN records 1044
current tsv file: hurricane_harvey_final_data.tsv
number of records 4443
counts
 severe_damage              556
mild_damage                222
little_or_no_damage        116
dont_know_or_cant_judge     22
Name: image_damage, dtype: int64
number of NaN records 3527
current tsv file: hurricane_irma_final_data.tsv
number of records 4525
counts
 severe_damage              320
little_or_no_damage        250
mild_damage                230
dont_know_or_cant_judge     19
Name: image_damage, dtype: int64
number of NaN records 3706
current tsv file: hurricane_maria_final_data.tsv
number of records 4562
counts
 severe_damage              509
mild_damage                273
little_or_no_damage         80
dont_know_or_cant_judge     42